In [1]:
from pyspark.sql import SparkSession


# Create a SparkSession
spark = SparkSession.builder.master("spark://spark-master:7077").getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
# Set the MinIO access key, secret key, endpoint, and other configurations
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "NzUmXT2E7YJIY3m9yFBz")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "5SCMGEtt52EziOcTUi9dyMt5dfFE2MBaK1ZaskPX")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
# Read a JSON file from an MinIO bucket using the access key, secret key, 
# and endpoint configured above
df = spark.read.option("header", "true") \
    .csv(f"s3a://raw/stops.txt")
# show data
df.show()

+-------+----------------+---------+----------+----------+
|stop_id|       stop_name|stop_desc|  stop_lat|  stop_lon|
+-------+----------------+---------+----------+----------+
|  18848|        Clínicas|     null|-23.554022|-46.671108|
|  18849|   Vila Madalena|     null|-23.546498|-46.691141|
|  18850|      Consolação|     null|-23.558094|-46.660205|
|  18851|       Conceição|     null|-23.635039|-46.641239|
|  18852|       Jabaquara|     null|-23.646033|-46.641028|
|  18853|       São Judas|     null|-23.625882|-46.640936|
|  18854|           Saúde|     null|-23.618245|-46.639139|
|  18855| Praça Da Árvore|     null|-23.610583|-46.637918|
|  18856|      Santa Cruz|     null|-23.598541|-46.636638|
|  18857|    Vila Mariana|     null|-23.589359|-46.634677|
|  18858|      Brigadeiro|     null|-23.567615|-46.649027|
|  18859|    Trianon-masp|     null|-23.563570|-46.653893|
|  18860|        Ana Rosa|     null|-23.581203|-46.638489|
|  18861|         Paraíso|     null|-23.575400|-46.64070

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark-master:7077").getOrCreate()
display(spark)

sc = spark.sparkContext

# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(100 + 1))
rdd.sum()

5050

In [7]:


from pyspark.sql import SparkSession
from pyspark.sql import Row

jar_files = [
    "/util/clickhouse-jdbc-0.3.2-all.jar"
]

# Initialize Spark session with JARs
spark = SparkSession.builder \
    .appName("test-jdbc") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", ",".join(jar_files)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","NzUmXT2E7YJIY3m9yFBz") \
    .config("spark.hadoop.fs.s3a.secret.key","5SCMGEtt52EziOcTUi9dyMt5dfFE2MBaK1ZaskPX") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


url = "jdbc:ch://clickhouse:8123/cameraaberta"
user = "admin" 
password = "admin"  
driver = "com.clickhouse.jdbc.ClickHouseDriver"

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stops.txt")
# show data
df.show()


df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "stops") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (stop_id)") \
    .mode("overwrite") \
    .save()



+-------+----------------+---------+----------+----------+
|stop_id|       stop_name|stop_desc|  stop_lat|  stop_lon|
+-------+----------------+---------+----------+----------+
|  18848|        Clínicas|     null|-23.554022|-46.671108|
|  18849|   Vila Madalena|     null|-23.546498|-46.691141|
|  18850|      Consolação|     null|-23.558094|-46.660205|
|  18851|       Conceição|     null|-23.635039|-46.641239|
|  18852|       Jabaquara|     null|-23.646033|-46.641028|
|  18853|       São Judas|     null|-23.625882|-46.640936|
|  18854|           Saúde|     null|-23.618245|-46.639139|
|  18855| Praça Da Árvore|     null|-23.610583|-46.637918|
|  18856|      Santa Cruz|     null|-23.598541|-46.636638|
|  18857|    Vila Mariana|     null|-23.589359|-46.634677|
|  18858|      Brigadeiro|     null|-23.567615|-46.649027|
|  18859|    Trianon-masp|     null| -23.56357|-46.653893|
|  18860|        Ana Rosa|     null|-23.581203|-46.638489|
|  18861|         Paraíso|     null|  -23.5754|  -46.640

In [39]:
df = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "example_table10") \
    .load()

df.show()


+---+----+
| id|name|
+---+----+
| 11|John|
| 12| Doe|
+---+----+

